# 1 - Import libraries

In [17]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [18]:
# Prepare the function for "cleaning" the texts
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

# 2 - Preparing datasets

## 2.1 - Preparing fake_or_real_news dataset 
### output:
#### data_fake_or_real_news - data
#### labels_fake_or_real_news - answers

In [19]:
# Import fake_or_real_news dataset
fake_or_real_ds = pd.read_csv('/home/veniamin/Documents/Juputer DS/CBS_proj/fake_or_real_news.csv', delimiter=',', header=0)
# Take only necessary colomns
fake_or_real_ds = fake_or_real_ds[['text','label']]


In [20]:
# Show fake_or_real_news dataset
fake_or_real_ds.head()

text label
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE
2  U.S. Secretary of State John F. Kerry said Mon...  REAL
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE
4  It's primary day in New York and front-runners...  REAL

In [21]:
# Create a list of "answers" and convert answers in 0 and 1
labels_fake_or_real_news = fake_or_real_ds['label'].map(lambda x : 1 if x == "REAL" else 0)

In [22]:
# Show "answers" for fake_or_real_news 
labels_fake_or_real_news.head()

0    0
1    0
2    1
3    0
4    1
Name: label, dtype: int64

In [23]:
# Clean the fake_or_real_news dataset
fake_or_real_ds['text'] = fake_or_real_ds['text'].map(lambda x: clean_text(x))

In [24]:
# Show the cleaned fake_or_real_news dataset
fake_or_real_ds.head(4)

text label
0  daniel greenfield shillman journal fellow free...  FAKE
1  googl pinterest digg linkedin reddit stumbleup...  FAKE
2  u s secretari state john kerri said monday sto...  REAL
3  kayde king kaydeek novemb 2016 lesson tonight ...  FAKE

In [25]:
# Check the balance of set
classes=fake_or_real_ds['label']
print(classes.value_counts())

REAL    3171
FAKE    3164
Name: label, dtype: int64


In [26]:
# Tokenize
vocabulary_size = 20000 
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(fake_or_real_ds['text'])

sequences = tokenizer.texts_to_sequences(fake_or_real_ds['text'])
# choose the length of sequences of tokens
data_fake_or_real_news = pad_sequences(sequences, maxlen=5000)

In [27]:
# Check the shape of final table
print(data_fake_or_real_news.shape)

(6335, 5000)


## 2.2 - Preparing test dataset from Liar Liar ...
### output:
#### data_test - data
#### labels_test - answers

In [12]:
# Import test dataset
test_ds = pd.read_csv('/home/veniamin/Dropbox/Data Science Projects (2018-2019)/Fake News/datasets/LIAR_dataset/test.tsv', delimiter='\t', header=None)
# Take only necessary colomns
test_ds = test_ds[[1,2]]
# Take only necessary rows
test_ds = test_ds.loc[test_ds[1].isin(['true', 'false']), 1:2]

In [13]:
# Show 'test' dataset
test_ds.head()

1                                                  2
0   true  Building a wall on the U.S.-Mexico border will...
1  false  Wisconsin is on pace to double the number of l...
2  false  Says John McCain has done nothing to help the ...
5   true  Over the past five years the federal governmen...
6   true  Says that Tennessee law requires that schools ...

In [14]:
# Create a list of "answers" and convert answers in 0 and 1
labels_test = test_ds[1].map(lambda x : 1 if x == "true" else 0)

In [15]:
# Show "answers" for test
labels_test.head()

0    1
1    0
2    0
5    1
6    1
Name: 1, dtype: int64

In [16]:
# Clean the test dataset
test_ds[2] = test_ds[2].map(lambda x: clean_text(x))

In [17]:
# Show the cleaned fake_or_real_news dataset
test_ds.head(4)

1                                                  2
0   true  build wall american - mexico border take liter...
1  false            wisconsin pace doubl number layoff year
2  false                 say john mccain done noth help vet
5   true  past five year feder govern paid 601 million r...

In [18]:
# Check the balance of set
classes=test_ds[1]
print(classes.value_counts())

false    249
true     208
Name: 1, dtype: int64


In [19]:
# Tokenize
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(test_ds[2])

sequences = tokenizer.texts_to_sequences(test_ds[2])
# choose the length of sequences of tokens
data_test = pad_sequences(sequences, maxlen=5000)

## 2.3 - Preparing train dataset from Liar Liar ...
### output:
#### data_train - data
#### labels_train - answers

In [20]:
# Import train dataset
train_ds = pd.read_csv('/home/veniamin/Dropbox/Data Science Projects (2018-2019)/Fake News/datasets/LIAR_dataset/train.tsv', delimiter='\t', header=None)
# Take only necessary colomns
train_ds = train_ds[[1,2]]
# Take only necessary rows
train_ds = train_ds.loc[train_ds[1].isin(['true', 'false']), 1:2]

In [21]:
# Show train dataset
train_ds.head()

1                                                  2
0   false  Says the Annies List political group supports ...
3   false  Health care reform legislation is likely to ma...
5    true  The Chicago Bears have had more starting quart...
12  false  When Mitt Romney was governor of Massachusetts...
16   true  McCain opposed a requirement that the governme...

In [22]:
train_ds.columns

Int64Index([1, 2], dtype='int64')

In [23]:
idxs=train_ds[1] == 1
len(idxs)
#train_ds[idxs[:300]]

3671

In [24]:
# Create a list of "answers" and convert answers in 0 and 1
labels_train = train_ds[1].map(lambda x : 1 if x == "true" else 0)

In [25]:
# Show "answers" for train
labels_train.head()

0     0
3     0
5     1
12    0
16    1
Name: 1, dtype: int64

In [26]:
# Clean the train dataset
train_ds[2] = train_ds[2].map(lambda x: clean_text(x))

In [27]:
# Show the cleaned train dataset
train_ds.head(4)

1                                                  2
0   false  say anni list polit group support third - trim...
3   false  health care reform legisl like mandat free sex...
5    true  chicago bear start quarterback last year total...
12  false  mitt romney governor massachusett didnt slow r...

In [28]:
# Check the balance of set
classes=train_ds[1]
print(classes.value_counts())

false    1995
true     1676
Name: 1, dtype: int64


In [29]:
# Tokenize
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_ds[2])
# choose the length of sequences of tokens
sequences = tokenizer.texts_to_sequences(train_ds[2])
data_train = pad_sequences(sequences, maxlen=5000)

# 2 - Models

## 2.1 - LSTM

### 2.1.1 - LSTM with fake_or_real_news

In [11]:
#set parametrs
model_lstm_forn = Sequential()
model_lstm_forn.add(Embedding(20000, 100, input_length=5000))
model_lstm_forn.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm_forn.add(Dense(1, activation='sigmoid'))
model_lstm_forn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.1.2 - LSTM with train from liar liar

In [32]:
#set parametrs
model_lstm_train = Sequential()
model_lstm_train.add(Embedding(20000, 100, input_length=5000))
model_lstm_train.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm_train.add(Dense(1, activation='sigmoid'))
model_lstm_train.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## 2.2 - LSTM with pre-traind weights from GLOVES

In [29]:
embeddings_index = dict()
f = open('/home/veniamin/Documents/Juputer DS/CBS_proj/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [30]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

### 2.2.1 - LSTM with pre-traind weights from GLOVES and trained on fake_or_real_news data set

In [14]:
model_glove_forn = Sequential()
model_glove_forn.add(Embedding(vocabulary_size, 100, input_length=5000, weights=[embedding_matrix], trainable=False))
model_glove_forn.add(Dropout(0.2))
model_glove_forn.add(Conv1D(64, 5, activation='relu'))
model_glove_forn.add(MaxPooling1D(pool_size=4))
model_glove_forn.add(LSTM(100))
model_glove_forn.add(Dense(1, activation='sigmoid'))
model_glove_forn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.2.2 - LSTM with pre-traind weights from GLOVES and trained on train data set from liar liar

In [33]:
model_glove_train = Sequential()
model_glove_train.add(Embedding(vocabulary_size, 100, input_length=5000, weights=[embedding_matrix], trainable=False))
model_glove_train.add(Dropout(0.2))
model_glove_train.add(Conv1D(64, 5, activation='relu'))
model_glove_train.add(MaxPooling1D(pool_size=4))
model_glove_train.add(LSTM(100))
model_glove_train.add(Dense(1, activation='sigmoid'))
model_glove_train.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## 2.3 Models for "synthetic" tests

### 2.3.1 - LSTM with fake_or_real_news (synthetic)

In [28]:
#set parametrs
model_lstm_forn_syn = Sequential()
model_lstm_forn_syn.add(Embedding(20000, 100, input_length=5000))
model_lstm_forn_syn.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm_forn_syn.add(Dense(1, activation='sigmoid'))
model_lstm_forn_syn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.3.2 - LSTM with pre-traind weights from GLOVES and trained on fake_or_real_news data set (synthetic)

In [31]:
model_glove_forn_syn = Sequential()
model_glove_forn_syn.add(Embedding(vocabulary_size, 100, input_length=5000, weights=[embedding_matrix], trainable=False))
model_glove_forn_syn.add(Dropout(0.2))
model_glove_forn_syn.add(Conv1D(64, 5, activation='relu'))
model_glove_forn_syn.add(MaxPooling1D(pool_size=4))
model_glove_forn_syn.add(LSTM(100))
model_glove_forn_syn.add(Dense(1, activation='sigmoid'))
model_glove_forn_syn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 3 - Training & Testing

## 3.1 - Training LSTM on fake_or_real_news

In [33]:
model_lstm_forn.fit(data_fake_or_real_news, np.array(labels_fake_or_real_news), epochs=3)# validation_split=0.4,

Epoch 1/3
6335/6335 [==============================] - 1039s 164ms/step - loss: 0.4639 - acc: 0.7973
Epoch 2/3
6335/6335 [==============================] - 1031s 163ms/step - loss: 0.2257 - acc: 0.9143
Epoch 3/3
6335/6335 [==============================] - 1021s 161ms/step - loss: 0.1194 - acc: 0.9620


### 3.1.1 - Testing on test dataset from Liar Liar

In [34]:
res_lstm_forn_on_test=model_lstm_forn.predict(data_test)

In [35]:
 #акк предсказания модели обученной на fake or real news и протестированной на liar liar tes
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test,res_lstm_forn_on_test.round()))

0.5295404814004376


### 3.1.2 - Testing on train dataset from Liar Liar

In [36]:
res_lstm_forn_on_train=model_lstm_forn.predict(data_train)

In [37]:
 #акк предсказания модели обученной на fake or real news и протестированной на liar liar tes
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_train,res_lstm_forn_on_train.round()))

0.5047670934350313


## 3.2 - Training LSTM on train dataset from Liar Liar

In [38]:
#обучаемся на liar liar train
model_lstm_train.fit(data_train, np.array(labels_train),epochs=3)# validation_split=0.4, 

Epoch 1/3
3671/3671 [==============================] - 595s 162ms/step - loss: 0.6847 - acc: 0.5538
Epoch 2/3
3671/3671 [==============================] - 602s 164ms/step - loss: 0.5751 - acc: 0.7093
Epoch 3/3
3671/3671 [==============================] - 646s 176ms/step - loss: 0.4074 - acc: 0.8172


### 3.2.1 - Testing on test dataset from Liar Liar

In [39]:
#тестируемся на liar liar test
res_lstm_train_on_test=model_lstm_train.predict(data_test)

In [40]:
 #акк предсказания модели обученной на liar liar train и протестированной на liar liar tes
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test,res_lstm_train_on_test.round()))

0.5164113785557987


### 3.2.2 - Testing on fake or real news dataset 

In [41]:
#тестируемся на fake or real
res_lstm_train_on_forn=model_lstm_train.predict(data_fake_or_real_news)

In [42]:
#акк предсказания модели обученной на liar liar train и протестированной на fake or real news

from sklearn.metrics import accuracy_score
print(accuracy_score(labels_fake_or_real_news,res_lstm_train_on_forn.round()))

0.47040252565114443


## 3.3 -   Training LSTM with pre-traind weights from GLOVES and fake_or_real_news data set

In [34]:
# using fake or real data set
model_glove_forn.fit(data_fake_or_real_news, np.array(labels_fake_or_real_news), epochs = 3) #validation_split=0.4,

Epoch 1/3
6335/6335 [==============================] - 377s 60ms/step - loss: 0.6368 - acc: 0.6314
Epoch 2/3
6335/6335 [==============================] - 375s 59ms/step - loss: 0.5222 - acc: 0.7479
Epoch 3/3
6335/6335 [==============================] - 375s 59ms/step - loss: 0.4679 - acc: 0.7830


### 3.3.1 - Testing on test dataset from liar liar

In [35]:
res_lstm_and_glove_on_forn_on_test=model_glove_forn.predict(data_test)

In [36]:
#акк предсказания модели обученной на liar liar train и протестированной на fake or real news

from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test,res_lstm_and_glove_on_forn_on_test.round()))

0.5273522975929978


### 3.3.2 - Testing on train dataset from liar liar

In [37]:
res_lstm_and_glove_on_forn_on_train=model_glove_forn.predict(data_train)

In [38]:
#акк предсказания модели обученной на liar liar train и протестированной на fake or real news

from sklearn.metrics import accuracy_score
print(accuracy_score(labels_train,res_lstm_and_glove_on_forn_on_train.round()))

0.5347316807409426


## 3.4 - Training LSTM with pre-traind weights from GLOVES and train data set from liar liar

In [39]:
# using fake or real data set
model_glove_train.fit(data_train, np.array(labels_train), epochs = 3) #validation_split=0.4,

Epoch 1/3
3671/3671 [==============================] - 214s 58ms/step - loss: 0.6867 - acc: 0.5576
Epoch 2/3
3671/3671 [==============================] - 213s 58ms/step - loss: 0.6653 - acc: 0.6031
Epoch 3/3
3671/3671 [==============================] - 213s 58ms/step - loss: 0.6368 - acc: 0.6336


### 3.4.1 - Testing on test dataset from liar liar

In [40]:
res_lstm_and_glove_on_train_on_test=model_glove_train.predict(data_test)

In [41]:
#акк предсказания модели обученной на liar liar train и протестированной на fake or real news

from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test,res_lstm_and_glove_on_train_on_test.round()))

0.5164113785557987


### 3.3.1 - Testing on fake or real news dataset 

In [42]:
res_lstm_and_glove_on_train_on_forn=model_glove_train.predict(data_fake_or_real_news)

In [43]:
#акк предсказания модели обученной на liar liar train и протестированной на fake or real news

from sklearn.metrics import accuracy_score
print(accuracy_score(labels_fake_or_real_news,res_lstm_and_glove_on_train_on_forn.round()))

0.5210734017363852


# 4 - Using fake_or_real_news dataset to improve accuracy (synthetically)

In [32]:
model_lstm_forn_syn.fit(data_fake_or_real_news, np.array(labels_fake_or_real_news),validation_split=0.2, epochs=3)# 

Train on 5068 samples, validate on 1267 samples
Epoch 1/3
5068/5068 [==============================] - 889s 175ms/step - loss: 0.4536 - acc: 0.7926 - val_loss: 0.3126 - val_acc: 0.8761
Epoch 2/3
5068/5068 [==============================] - 918s 181ms/step - loss: 0.1909 - acc: 0.9311 - val_loss: 0.2849 - val_acc: 0.8950
Epoch 3/3
5068/5068 [==============================] - 976s 193ms/step - loss: 0.1044 - acc: 0.9661 - val_loss: 0.3603 - val_acc: 0.8548


In [33]:
# using fake or real data set
model_glove_forn_syn.fit(data_fake_or_real_news, np.array(labels_fake_or_real_news),validation_split=0.2, epochs = 3) #,

Train on 5068 samples, validate on 1267 samples
Epoch 1/3
5068/5068 [==============================] - 329s 65ms/step - loss: 0.5375 - acc: 0.7245 - val_loss: 0.4451 - val_acc: 0.8051
Epoch 2/3
5068/5068 [==============================] - 323s 64ms/step - loss: 0.4153 - acc: 0.8133 - val_loss: 0.3752 - val_acc: 0.8421
Epoch 3/3
5068/5068 [==============================] - 328s 65ms/step - loss: 0.3593 - acc: 0.8540 - val_loss: 0.3082 - val_acc: 0.8721


In [35]:
# Split dataset into training and testing
from sklearn.model_selection import train_test_split

data_train_fake_or_real, data_test_fake_or_real, labels_train_fake_or_real, labels_test_fake_or_real = train_test_split(
    data_fake_or_real_news, labels_fake_or_real_news, test_size = 0.2, random_state = 0
)

In [36]:
#set parametrs
model_lstm_forn_syn_dam = Sequential()
model_lstm_forn_syn_dam.add(Embedding(20000, 100, input_length=5000))
model_lstm_forn_syn_dam.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm_forn_syn_dam.add(Dense(1, activation='sigmoid'))
model_lstm_forn_syn_dam.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.3.2 - LSTM with pre-traind weights from GLOVES and trained on fake_or_real_news data set (synthetic)

In [37]:
model_glove_forn_syn_dam = Sequential()
model_glove_forn_syn_dam.add(Embedding(vocabulary_size, 100, input_length=5000, weights=[embedding_matrix], trainable=False))
model_glove_forn_syn_dam.add(Dropout(0.2))
model_glove_forn_syn_dam.add(Conv1D(64, 5, activation='relu'))
model_glove_forn_syn_dam.add(MaxPooling1D(pool_size=4))
model_glove_forn_syn_dam.add(LSTM(100))
model_glove_forn_syn_dam.add(Dense(1, activation='sigmoid'))
model_glove_forn_syn_dam.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model_lstm_forn_syn_dam.fit(data_train_fake_or_real, np.array(labels_train_fake_or_real),epochs=3)# 

Epoch 1/3
5068/5068 [==============================] - 839s 166ms/step - loss: 0.4756 - acc: 0.7725
Epoch 2/3
5068/5068 [==============================] - 830s 164ms/step - loss: 0.1977 - acc: 0.9292
Epoch 3/3
5068/5068 [==============================] - 830s 164ms/step - loss: 0.1016 - acc: 0.9669


In [39]:
res_lstm_forn_on_test_syn_dam=model_lstm_forn_syn_dam.predict(data_test_fake_or_real)

In [40]:
 #акк предсказания модели обученной на fake or real news и протестированной на liar liar tes
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test_fake_or_real,res_lstm_forn_on_test_syn_dam.round()))

0.8768745067087609


In [41]:
# using fake or real data set
model_glove_forn_syn_dam.fit(data_train_fake_or_real, np.array(labels_train_fake_or_real), epochs = 3) #,

Epoch 1/3
5068/5068 [==============================] - 312s 62ms/step - loss: 0.5392 - acc: 0.7311
Epoch 2/3
5068/5068 [==============================] - 302s 60ms/step - loss: 0.4210 - acc: 0.8131
Epoch 3/3
5068/5068 [==============================] - 303s 60ms/step - loss: 0.4101 - acc: 0.8193


In [42]:
res_glove_forn_on_test_syn_dam=model_glove_forn_syn_dam.predict(data_test_fake_or_real)

In [43]:
 #акк предсказания модели обученной на fake or real news и протестированной на liar liar tes
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test_fake_or_real,res_glove_forn_on_test_syn_dam.round()))

0.7632202052091555
